In [1]:
import itertools
from scipy.optimize import linear_sum_assignment
import numpy as np
from tqdm import tqdm
from collections import Counter

In [2]:
import os 
os.environ['CPLEX_STUDIO_DIR201'] = "/opt/ibm/ILOG/CPLEX_Studio_Community201"
os.environ['CPLEX_STUDIO_KEY'] = "xxxx"
from docplex.mp.model import Model
from docplex.mp.environment import Environment

In [3]:
permutations = [''.join(x) for x in itertools.permutations(['1','2','3','4','5','6','7'], 7)]
permutations1 = ['12' + ''.join(x) for x in itertools.permutations(['3','4','5','6','7'], 5)]
permutations = permutations1 + permutations

In [4]:
N = len(permutations)
N

5160

In [5]:
MAX_COST = 3

In [6]:
p2i = {}
for i,p in enumerate(permutations):
    p2i[p] = i

In [7]:
len(permutations)

5160

In [8]:
permutations = np.array(permutations)
permutations_set = set(permutations)
#np.random.shuffle(permutations)

In [9]:
def get_cost(p1, p2):
    if p1 == p2:
        return 10000
    for i in range(1, MAX_COST+1):
        #if i == 2 and p1[0] == '1' and p1[2] == '2':
        #    return 10000
        if p1[i:] == p2[:-i]:
            return i 
    return 10000

p1, p2 = '1234567', '2345672'
get_cost(p1, p2)

1

In [10]:
def get_dmat(permutations):
    L = len(permutations)
    var_idx = []
    cost = {}
    for i,p1 in enumerate(tqdm(permutations)):
        for j,p2 in enumerate(permutations):
            cost_ij = get_cost(p1, p2)
            if cost_ij <= MAX_COST:
                var_idx.append((i, j))
                cost[(i,j)] = cost_ij
    return var_idx, cost

var_idx, all_cost = get_dmat(permutations)

len(var_idx)

100%|██████████████████████████████████████| 5160/5160 [00:39<00:00, 129.04it/s]


47520

In [11]:
wild_edges = []

for p in permutations1:
    p1 = p[4:6] + p[0] + p[6] + p[1:4]
    p2 = p[5] + p[0] + p[4] + p[1:4] + p[6]
    p5 = p2[1:] + p2[0]
    p7 = p5[0] + p5[2:] + p5[1]
    p3 = p[6] + p[5] + p[0] + p[4] + p[1:4]
    p4 = p[5] + p[0] + p[6] + p[1:5]
    p6 = p4[1:] + p4[0]
    p8 = p6[0] + p6[2:] + p6[1]
    if not (p3, p4, p6, p8, p1, p2, p5, p7) in wild_edges:
        wild_edges.append((p1, p2, p5, p7, p3, p4, p6, p8))
    
wild_edges

[('5617234',
  '6152347',
  '1523476',
  '1234765',
  '7615234',
  '6172345',
  '1723456',
  '1234567'),
 ('5716234',
  '7152346',
  '1523467',
  '1234675',
  '6715234',
  '7162345',
  '1623457',
  '1234576'),
 ('6517234',
  '5162347',
  '1623475',
  '1234756',
  '7516234',
  '5172346',
  '1723465',
  '1234657'),
 ('4617235',
  '6142357',
  '1423576',
  '1235764',
  '7614235',
  '6172354',
  '1723546',
  '1235467'),
 ('4716235',
  '7142356',
  '1423567',
  '1235674',
  '6714235',
  '7162354',
  '1623547',
  '1235476'),
 ('6417235',
  '4162357',
  '1623574',
  '1235746',
  '7416235',
  '4172356',
  '1723564',
  '1235647'),
 ('4517236',
  '5142367',
  '1423675',
  '1236754',
  '7514236',
  '5172364',
  '1723645',
  '1236457'),
 ('4715236',
  '7142365',
  '1423657',
  '1236574',
  '5714236',
  '7152364',
  '1523647',
  '1236475'),
 ('5417236',
  '4152367',
  '1523674',
  '1236745',
  '7415236',
  '4172365',
  '1723654',
  '1236547'),
 ('4516237',
  '5142376',
  '1423765',
  '1237654',
  '

In [12]:
len(wild_edges)

60

In [13]:
wild_idx = []

for (p1, p2, p5, p7, p3, p4, p6, p8) in wild_edges:
    (p1, p2, p5, p7, p3, p4, p6, p8) = (p2i[p1], p2i[p2], p2i[p5], p2i[p7], p2i[p3], p2i[p4], p2i[p6], p2i[p8])
    var_idx.append((p1, p2))
    var_idx.append((p3, p4))
    all_cost[(p1, p2)] = 1 - 0.001
    all_cost[(p3, p4)] = 1 - 0.001
    wild_idx.append((p1, p2, p5, p7, p3, p4, p6, p8))

In [14]:
wild2_idx = [(p1, p2, p5, p7) for (p1, p2, p5, p7, p3, p4, p6, p8) in wild_idx] + [(p3, p4, p6, p8) for (p1, p2, p5, p7, p3, p4, p6, p8) in wild_idx]
len(wild2_idx)

120

In [15]:
wild2_edges = [(permutations[i], permutations[j], permutations[k], permutations[l]) for i,j,k,l in wild2_idx]
wild2_edges

[('5617234', '6152347', '1523476', '1234765'),
 ('5716234', '7152346', '1523467', '1234675'),
 ('6517234', '5162347', '1623475', '1234756'),
 ('4617235', '6142357', '1423576', '1235764'),
 ('4716235', '7142356', '1423567', '1235674'),
 ('6417235', '4162357', '1623574', '1235746'),
 ('4517236', '5142367', '1423675', '1236754'),
 ('4715236', '7142365', '1423657', '1236574'),
 ('5417236', '4152367', '1523674', '1236745'),
 ('4516237', '5142376', '1423765', '1237654'),
 ('4615237', '6142375', '1423756', '1237564'),
 ('5416237', '4152376', '1523764', '1237645'),
 ('5617243', '6152437', '1524376', '1243765'),
 ('5716243', '7152436', '1524367', '1243675'),
 ('6517243', '5162437', '1624375', '1243756'),
 ('3617245', '6132457', '1324576', '1245763'),
 ('3716245', '7132456', '1324567', '1245673'),
 ('6317245', '3162457', '1624573', '1245736'),
 ('3517246', '5132467', '1324675', '1246753'),
 ('3715246', '7132465', '1324657', '1246573'),
 ('5317246', '3152467', '1524673', '1246735'),
 ('3516247', 

In [16]:
def get_pair_idx(var_idx):
    firsts = {}
    lasts = {}
    for i,j in var_idx:
        if not firsts.get(j, False):
            firsts[j] = [i]
        else:
            firsts[j].append(i)
        if not lasts.get(i, False):
            lasts[i] = [j]
        else:
            lasts[i].append(j)
    return firsts, lasts


firsts, lasts = get_pair_idx(var_idx)

In [17]:
len(firsts[0]), len(lasts[0])

(9, 9)

In [18]:
len(firsts[120]), len(lasts[120])

(9, 9)

In [19]:
env = Environment()
env.print_information()

mdl = Model("santa")
next_perm = mdl.binary_var_dict(var_idx)
start_perm = mdl.binary_var_list(range(N), name='start')
end_perm = mdl.binary_var_list(range(N), name='end')
class_perm = mdl.continuous_var_matrix(range(N), range(3), lb=0, ub=1, name='class')
class_perm_binary = mdl.binary_var_matrix(range(240), range(3), name='class_binary')
mdl.print_information()

* system is: Linux 64bit
* Python version 3.8.12, located at: /home/jpuget/miniconda3/envs/cplex201/bin/python
* docplex is present, version is 2.18.200
* CPLEX library is present, version is 20.1.0.0, located at: /home/jpuget/miniconda3/envs/cplex201/lib/python3.8/site-packages
* pandas is present, version is 1.3.4
Model: santa
 - number of variables: 74160
   - binary=58680, integer=0, continuous=15480
 - number of constraints: 0
   - linear=0
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [20]:
mdl.add_constraints(
    mdl.sum(next_perm[(i,j)] for i in firsts[j]) + start_perm[j] == 1
    for j in range(N))

mdl.add_constraints(
    mdl.sum(next_perm[(i,j)] for j in lasts[i]) + end_perm[i] == 1
    for i in range(N))

mdl.print_information()

Model: santa
 - number of variables: 74160
   - binary=58680, integer=0, continuous=15480
 - number of constraints: 10320
   - linear=10320
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [21]:
mdl.add_constraint(mdl.sum(start_perm) == mdl.sum(end_perm))
mdl.add_constraint(mdl.sum(start_perm) == 90)
#mdl.add_constraints(start_perm[i] <= start_perm[1+120] for i in range(120))

mdl.print_information()

Model: santa
 - number of variables: 74160
   - binary=58680, integer=0, continuous=15480
 - number of constraints: 10322
   - linear=10322
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [22]:
# from model_045 solution

mdl.add_constraints(start_perm[i] == 0 for i,p in enumerate(permutations) if not p.startswith('12'))
mdl.add_constraints(end_perm[i] == 0 for i,p in enumerate(permutations) if (p[0] != '1' and p[2] != '2'))

mdl.print_information()

Model: santa
 - number of variables: 74160
   - binary=58680, integer=0, continuous=15480
 - number of constraints: 18962
   - linear=18962
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [23]:
# next_perm[(i,j)] == 1 => (class_perm[i] == class_perm[j])

mdl.add_constraints(
    class_perm[(i, k)] - class_perm[(j, k)] <= (1 - next_perm[(i,j)])
    for i,j in var_idx
    for k in range(3)
)

mdl.add_constraints(
    class_perm[(j, k)] - class_perm[(i, k)] <= (1 - next_perm[(i,j)])
    for i,j in var_idx
    for k in range(3)
)

mdl.add_constraints(
    class_perm_binary[(i+120, k)] + class_perm_binary[(i, k)] <= 1
    for i in range(120)
    for k in range(3)
)

mdl.add_constraints(
    class_perm[(i,k)] == class_perm_binary[(i,k)]
    for i in range(240)
    for k in range(3)
)

mdl.add_constraints(
    mdl.sum(class_perm[(i,k)] for k in range(3)) == 1
    for i in range(240)
)

mdl.add_constraints(
    mdl.sum(class_perm[(i,k)] for i in range(N)) == N // 3
    for k in range(3)
)

mdl.add_constraints(
    mdl.sum(class_perm[(i,k)] for i in range(240)) == 240 // 3
    for k in range(3)
)

mdl.print_information()

Model: santa
 - number of variables: 74160
   - binary=58680, integer=0, continuous=15480
 - number of constraints: 306128
   - linear=306128
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [24]:
def rot(s, i=1):
    if i <= 0:
        return s
    return s[i:] + s[:i]

def get_canon(s):
    best = s
    for i in range(len(s)-1):
        s = rot(s)
        if s[0] < best[0]:
            best = s
    return best

In [25]:
def get_1_cycle(i, s):
    cycle = [rot(s, i) for i in range(len(s))]
    cycle = [p2i[p] for p in cycle]
    cycle[0] = i
    return cycle
    
cycles1 = []
for i,p in enumerate(permutations):
    if p != get_canon(p):
        continue
    cycles1.append(get_1_cycle(i, p))

def get_alternate(i):
    if i < 120:
        return i + 120
    elif i < 240:
        return i - 120
    return i

def get_inner(cycle):
    inner = []
    for j in cycle:
        inner.extend([(i, j) for i in firsts[j] if i not in cycle and get_alternate(i) not in cycle])
    return inner

def get_outer(cycle):
    outer = []
    for i in cycle:
        outer.extend([(i, j) for j in lasts[i] if j not in cycle and get_alternate(j) not in cycle])
    return outer

var_idx_set = set(var_idx)

def get_cycle_edges(cycle):
    edges = [(i,j) for i,j in itertools.permutations(cycle, 2) if (i,j) in var_idx_set]
    return edges
    

In [26]:
mdl.add_constraints(
    mdl.sum(next_perm[(i,j)] for i,j in get_inner(cycle)) + mdl.sum(start_perm[j] for j in cycle) >= 1
    for cycle in cycles1
)

mdl.add_constraints(
    mdl.sum(next_perm[(i,j)] for i,j in get_outer(cycle)) + mdl.sum(end_perm[i] for i in cycle) >= 1
    for cycle in cycles1
)

#mdl.add_constraints(
#    mdl.sum(all_cost[(i,j)] * next_perm[(i,j)] for i,j in get_cycle_edges(cycle)) <= len(cycle) - 1
#    for cycle in cycles1
#)

mdl.print_information()

Model: santa
 - number of variables: 74160
   - binary=58680, integer=0, continuous=15480
 - number of constraints: 307808
   - linear=307808
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [27]:
def remove_start(s, start):
    idx = s.index(start)
    s = s[idx:] + s[:idx]
    return s[0] + get_canon(s[1:])

def get_2_cycle(p, start):
    p = remove_start(p, start)
    p1 = p[1:]
    inner = [rot(p1, i) for i in range(len(p1))]
    cycle = [rot(q+start, j) for q in inner for j in range(len(p))]
    #cycle1 = [get_1_cycle(q+start) for q in inner]
    #cycle = get_canon(cycle)
    cycle = [p2i[p] for p in cycle]
    return p, cycle

cycles2 = {}

for p in tqdm(permutations[120:]):
    for start in p:
        p1 = remove_start(p, start)
        if cycles2.get(p1, None) is not None:
            continue
        p1, cycle = get_2_cycle(p, start)
        cycles2[p1] = cycle

cycles2['2134567']

len(cycles2)

def get_alternate_2cycles(cycle):
    mandatory = [i for i in cycle if i >= 120 and i < 240]
    if len(mandatory) == 0:
        return [cycle]
    if len(mandatory) == 1:
        new_cycle = [(i if i >= 240 else i - 120) for i in cycle]
        return [cycle, new_cycle]
    patterns = [p for p in itertools.product([0,1], repeat=5)]
    new_cycles = []
    for pattern in patterns:
        new_cycle = []
        for i in cycle:
            if i >= 240:
                new_cycle.append(i)
            else:
                if pattern[0]:
                    new_cycle.append(i - 120)
                else:
                    new_cycle.append(i)
                pattern = pattern[1:]
        new_cycles.append(new_cycle)
    return new_cycles

def get_alternate_2cycles(cycle):
    new_cycle = []
    duplicate = False
    for i in cycle:
        ii = get_alternate(i)
        if i != ii:
            duplicate = True
        new_cycle.append(ii)
    if duplicate:
        return [cycle, new_cycle]
    else:
        return [cycle]

all_cycles2 = [cycle for v in cycles2.values() for cycle in get_alternate_2cycles(v)]
len(all_cycles2)

100%|████████████████████████████████████| 5040/5040 [00:00<00:00, 34120.89it/s]


1200

In [28]:
mdl.add_constraints(
    mdl.sum(next_perm[(i,j)] for i,j in get_inner(cycle)) + mdl.sum(start_perm[j] for j in cycle) >= 1
    for cycle in all_cycles2
)

mdl.add_constraints(
    mdl.sum(next_perm[(i,j)] for i,j in get_outer(cycle)) + mdl.sum(end_perm[i] for i in cycle) >= 1
    for cycle in all_cycles2
)

mdl.print_information()

Model: santa
 - number of variables: 74160
   - binary=58680, integer=0, continuous=15480
 - number of constraints: 310208
   - linear=310208
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [29]:
N3 = N // 3
#N3 = 108

u = mdl.continuous_var_list(range(N), lb=0, ub=N3-1, name='u')

mdl.add_constraints(
    u[i] - u[j] + N3 * next_perm[(i,j)] <= N3 - 1
    for i,j in var_idx 
)

mdl.add_constraints(
    u[i] <= (N3 - 1) * (1 - start_perm[i])
    for i in range(N)
)

mdl.add_constraints(
    u[i] >= (1 - start_perm[i])
    for i in range(N)
)

mdl.print_information()

Model: santa
 - number of variables: 79320
   - binary=58680, integer=0, continuous=20640
 - number of constraints: 368168
   - linear=368168
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [30]:
s0 = '21111112111111211111121111112111111'
#s1 = '111121' + s0
def get_string(start, code):
    string = start
    seq = [p2i[start]]
    for c in code:
        if c == '1':
            start = rot(start)
            string = string + start[-1]
            seq.append(p2i[start])
        elif c == '2':
            start = rot(start, 2)
            start = start[:5] + start[-1] + start[-2]
            string = string + start[-2:]
            seq.append(p2i[start])
        elif c == '3':
            start = rot(start, 3)            
            start = start[:4] + start[-1] + start[-3] + start[-2]
            string = string + start[-3:]
            seq.append(p2i[start])
    return seq, string

In [31]:
idx, string = get_string('1234567', s0)
permutations[idx], string

(array(['1234567', '3456721', '4567213', '5672134', '6721345', '7213456',
        '2134567', '1345672', '4567231', '5672314', '6723145', '7231456',
        '2314567', '3145672', '1456723', '5672341', '6723415', '7234156',
        '2341567', '3415672', '4156723', '1567234', '6723451', '7234516',
        '2345167', '3451672', '4516723', '5167234', '1672345', '7234561',
        '2345617', '3456172', '4561723', '5617234', '6172345', '1723456'],
       dtype='<U7'),
 '12345672134567231456723415672345167234561723456')

In [32]:
for k in range(120):
    seq, _ = get_string(permutations[k], s0)
    mdl.add_constraints(
        next_perm[(i,j)] == 1 for i,j in zip(seq[:-3], seq[1:-2])
    )
    mdl.add_constraint(
        next_perm[(seq[-2], seq[-1])] == 1
    )

mdl.print_information()

Model: santa
 - number of variables: 79320
   - binary=58680, integer=0, continuous=20640
 - number of constraints: 372248
   - linear=372248
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [33]:
mdl.add_constraint(
    mdl.sum(next_perm[(p1, p2)] for (p1, p2, p5, p7) in wild2_idx) == 6
)

mdl.add_constraints(
    next_perm[(p1, p2)] == next_perm[(p3, p4)]
    for (p1, p2, p5, p7, p3, p4, p6, p8) in wild_idx
)

mdl.add_constraints(
    next_perm[(p1, p2)] <= end_perm[p5]
    for (p1, p2, p5, p7) in wild2_idx
)

mdl.print_information()

Model: santa
 - number of variables: 79320
   - binary=58680, integer=0, continuous=20640
 - number of constraints: 372429
   - linear=372429
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [34]:
wild_class_idx = [(p1, p2, p5, p7, k) for k in range(3) for (p1, p2, p5, p7) in wild2_idx]

wild_class = mdl.continuous_var_dict(wild_class_idx, lb=0, ub=1, name='wild_class')

In [35]:
wild_class_idx

[(3498, 3792, 481, 125, 0),
 (3618, 4512, 480, 123, 0),
 (4218, 3072, 601, 124, 0),
 (2778, 3768, 361, 131, 0),
 (2898, 4488, 360, 129, 0),
 (4098, 2352, 603, 130, 0),
 (2658, 3048, 363, 137, 0),
 (2892, 4489, 362, 135, 0),
 (3378, 2328, 483, 136, 0),
 (2652, 3049, 365, 143, 0),
 (2772, 3769, 364, 141, 0),
 (3372, 2329, 485, 142, 0),
 (3499, 3794, 487, 149, 0),
 (3619, 4514, 486, 147, 0),
 (4219, 3074, 607, 148, 0),
 (2058, 3744, 241, 155, 0),
 (2178, 4464, 240, 153, 0),
 (3978, 1632, 609, 154, 0),
 (1938, 3024, 243, 161, 0),
 (2172, 4465, 242, 159, 0),
 (3258, 1608, 489, 160, 0),
 (1932, 3025, 245, 167, 0),
 (2052, 3745, 244, 165, 0),
 (3252, 1609, 491, 166, 0),
 (2779, 3770, 367, 173, 0),
 (2899, 4490, 366, 171, 0),
 (4099, 2354, 613, 172, 0),
 (2059, 3746, 247, 179, 0),
 (2179, 4466, 246, 177, 0),
 (3979, 1634, 615, 178, 0),
 (1818, 2304, 249, 185, 0),
 (2166, 4467, 248, 183, 0),
 (2538, 1584, 369, 184, 0),
 (1812, 2305, 251, 191, 0),
 (2046, 3747, 250, 189, 0),
 (2532, 1585, 371, 1

In [36]:
mdl.add_constraints(
    wild_class[(p1, p2, p5, p7, k)] <= class_perm[(p1, k)]
    for (p1, p2, p5, p7, k) in wild_class_idx
)

mdl.add_constraints(
    wild_class[(p1, p2, p5, p7, k)] + class_perm[(p7, k)] <= 1
    for (p1, p2, p5, p7, k) in wild_class_idx
)

mdl.add_constraints(
    wild_class[(p1, p2, p5, p7, k)] + class_perm[(p7 - 120, k)] <= 1
    for (p1, p2, p5, p7, k) in wild_class_idx
)

mdl.add_constraints(
    wild_class[(p1, p2, p5, p7,  k)] <= next_perm[(p1, p2)]
    for (p1, p2, p5, p7, k) in wild_class_idx
)

mdl.add_constraints(
    mdl.sum(wild_class[(p1, p2, p5, p7, k)] for (p1, p2, p5, p7) in wild2_idx) == 2
    for k in range(3)
)
mdl.print_information()

Model: santa
 - number of variables: 79680
   - binary=58680, integer=0, continuous=21000
 - number of constraints: 373872
   - linear=373872
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [37]:
objective = mdl.sum(next_perm[idx] * all_cost[idx] for idx in var_idx) + 6.9999 * mdl.sum(start_perm)
                          
# Set objective function
mdl.minimize(objective)

mdl.print_information()

mdl.parameters.mip.tolerances.mipgap = 0.00
mdl.parameters.timelimit = 100000 
mdl.parameters.threads = 20
#mdl.parameters.mip.cuts.gomory = -1
mdl.parameters.mip.tolerances.uppercutoff = 7320 - 7*120 + 1e-3

mdl.solve(log_output=True)
mdl.report()
  

Model: santa
 - number of variables: 79680
   - binary=58680, integer=0, continuous=21000
 - number of constraints: 373872
   - linear=373872
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP
Checking license ...
License found. [1.63 s]
Version identifier: 20.1.0.0 | 2020-11-11 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 20
CPXPARAM_RandomSeed                              202001241
CPXPARAM_TimeLimit                               100000
CPXPARAM_MIP_Tolerances_UpperCutoff              6480.0010000000002
CPXPARAM_MIP_Tolerances_MIPGap                   0
Tried aggregator 6 times.
MIP Presolve eliminated 302400 rows and 50400 columns.
MIP Presolve modified 84481 coefficients.
Aggregator did 17220 substitutions.
Reduced MIP has 54252 rows, 12060 columns, and 208740 nonzeros.
Reduced MIP has 8100 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.72 sec. (677.23 ticks)
Probing fixed 0 v

In [38]:
sol = mdl.solution
sol

s = sol.get_value_dict(next_perm)
s
s_start = sol.get_value_list(start_perm)
s_end = sol.get_value_list(end_perm)
s_u = sol.get_value_list(u)
s_class = sol.get_value_dict(class_perm)

import pickle as pkl

with open('../subs/santa_103.pkl', 'wb') as file:
    pkl.dump((s, s_start, s_end, s_u, s_class), file)

In [39]:
import pickle as pkl

with open('../subs/santa_103.pkl', 'rb') as file:
    (s, s_start, s_end, s_u, s_class) = pkl.load(file)

In [40]:
s_cost = {}
s_cost1 = {}
s_sol = {}
for k,v in s.items():
    if v > 0:
        s_cost[k] = all_cost[k] * v
        s_cost1[k] = all_cost[k] 
        s_sol[k] = v

s_cost



C = Counter(s_cost.values())
C1 = Counter(s_cost1.values())
Cs = Counter(s_sol.values())
C, C1, Cs, Counter(s_start), Counter(s_end), Counter(s_class.values())

(Counter({1.0: 4314, 2.0: 720, 3.0: 30, 0.999: 6}),
 Counter({1: 4314, 2: 720, 3: 30, 0.999: 6}),
 Counter({1.0: 5070}),
 Counter({0: 5070, 1.0: 90}),
 Counter({0: 5070, 1.0: 90}),
 Counter({0: 10266,
          1.0: 5134,
          8.026912468039882e-13: 4,
          -8.026912468039882e-13: 1,
          5.837649880906799e-13: 9,
          -5.837649880906799e-13: 21,
          -1.3864465131518955e-12: 3,
          1.3864465131518955e-12: 5,
          1.0000000000019702: 2,
          1.1571854585668007e-12: 3,
          0.9999999999994162: 12,
          1.0000000000005838: 11,
          -1.1675299761813599e-12: 3,
          1.9702115012425755e-12: 2,
          -5.837552663479073e-13: 3,
          0.9999999999988325: 1}))

In [41]:
for (i,j), v in s_cost1.items():
    if v==3:
        print(permutations[i], permutations[j])

1326754 6754123
1327564 7564123
1327654 7654123
1426735 6735124
1427365 7365124
1427635 7635124
1523467 3467125
1523647 3647125
1524367 4367125
1524637 4637125
1526347 6347125
1526437 6437125
1526743 6743125
1527463 7463125
1527643 7643125
1623475 3475126
1623574 3574126
1623745 3745126
1624375 4375126
1624573 4573126
1624753 4753126
1625374 5374126
1625473 5473126
1625734 5734126
1723456 3456127
1723546 3546127
1724356 4356127
1724536 4536127
1725346 5346127
1725436 5436127


In [42]:
p_class = [int(np.round(s_class[(i, 1)] + 2 * s_class[(i, 2)]) )for i in range(N)]
Counter(p_class)

Counter({1: 1720, 0: 1720, 2: 1720})

In [43]:
Counter(p_class[:240])

Counter({1: 80, 0: 80, 2: 80})

In [44]:
costs = np.zeros(3)
for (i,j), c in s_cost1.items():
    if (p_class[i] != p_class[j]):
        print(p_class[i], p_class[j])
    costs[(p_class[i])] += c
for i in range(N):
    if s_start[i] > 0:
        costs[(p_class[i])] += 7
costs

array([2159.998, 2159.998, 2159.998])

In [45]:
def get_cycle(start, s_sol, s_cost, s_start, s_end):
    if s_start[start]:
        cycle_cost = 7
    else:
        cycle_cost = 0
    cycle = [start]
    prev = start
    while True:
        if s_end[prev]:
            return cycle, cycle_cost
        candidates0 = []
        for j in lasts[prev]:
            if s_sol.get((prev, j), 0) > 0:
                candidates0.append(j)
        candidates1 = [j for j in candidates0 if j not in cycle]
        if len(candidates1) > 0:
            candidates = candidates1
        else:
            return cycle, cycle_cost
        best_cost = 10000
        best_j = -1
        for j in candidates:
            cost = s_cost[(prev, j)]
            if cost < best_cost:
                best_cost = cost
                best_j = j
        cycle.append(best_j)
        cycle_cost = cycle_cost + s_cost[(prev, best_j)]
        s_sol[(prev, best_j)] = s_sol[(prev, best_j)] - 1
        prev = best_j    
    return cycle, cycle_cost
        

In [46]:
all_cycles = [[], [], []]
remaining = 1e6
mandatory_start = [i for i in range(N) if s_start[i]]
s_sol1 = s_sol.copy()

for k in mandatory_start:
    cycle, cost =  get_cycle(k, s_sol1, s_cost1, s_start, s_end)    
    Cs = Counter(s_sol1.values())
    remaining = np.sum([k*v for k,v in Cs.items()])    
    all_cycles[p_class[k]].append(cycle)
    print(len(cycle), cost, remaining, (len(cycle) - 1) // 36, (len(cycle) - 1) % 36, Cs)
    if remaining == 0:
        break

Cs = Counter(s_sol1.values())
Cs

82 101 4989.0 2 9 Counter({1.0: 4989, 0.0: 81})
146 176 4844.0 4 1 Counter({1.0: 4844, 0.0: 226})
82 100.999 4763.0 2 9 Counter({1.0: 4763, 0.0: 307})
82 101 4682.0 2 9 Counter({1.0: 4682, 0.0: 388})
94 115 4589.0 2 21 Counter({1.0: 4589, 0.0: 481})
54 68 4536.0 1 17 Counter({1.0: 4536, 0.0: 534})
94 115 4443.0 2 21 Counter({1.0: 4443, 0.0: 627})
42 54 4402.0 1 5 Counter({1.0: 4402, 0.0: 668})
82 101 4321.0 2 9 Counter({1.0: 4321, 0.0: 749})
82 101 4240.0 2 9 Counter({1.0: 4240, 0.0: 830})
82 101 4159.0 2 9 Counter({1.0: 4159, 0.0: 911})
82 101 4078.0 2 9 Counter({1.0: 4078, 0.0: 992})
42 54 4037.0 1 5 Counter({1.0: 4037, 0.0: 1033})
48 61 3990.0 1 11 Counter({1.0: 3990, 0.0: 1080})
106 129 3885.0 2 33 Counter({1.0: 3885, 0.0: 1185})
82 101 3804.0 2 9 Counter({1.0: 3804, 0.0: 1266})
78 96 3727.0 2 5 Counter({1.0: 3727, 0.0: 1343})
134 162 3594.0 3 25 Counter({1.0: 3594, 0.0: 1476})
54 68 3541.0 1 17 Counter({1.0: 3541, 0.0: 1529})
54 68 3488.0 1 17 Counter({1.0: 3488, 0.0: 1582})
48 61

Counter({0.0: 5070})

In [47]:
[np.sum([len(cycle) for cycle in all_cycles[i]]) for i in range(3)]

[1720, 1720, 1720]

In [50]:
def get_concat(perms):
    string = permutations[perms[0]]
    wild = False
    for j in perms[1:]:
        p = permutations[j]
        if p == string[-7:]:
            string = string + p
            continue
        for i in range(1, 8):
            if string[i-7:] == p[:-i]:
                break
        if i < 7:
            string = string + p[-i:]
        else:
            p1 = p[1:] + p[0]
            p2 = p1[0] + p1[2:] + p1[1]
            if (string[-7:], p, p1, p2) in wild2_edges:
                print(string[-7:], p)
                string = string[:-6] + p 
                wild = True
            else:
                string = string + p
    return string, wild

In [51]:
cats = [[get_concat(cycle) for cycle in all_cycles[i]] for i in range(3)]
[len(cat) for cat in cats]

6713245 7162453
6415273 4162735
4713256 7142563
3716245 7132456
5416273 4152736
3714256 7132564


[30, 30, 30]

In [52]:
[np.sum([len(cat[0]) for cat in catsi]) for catsi in cats]

[2160, 2160, 2160]

In [53]:
[len([cat for cat,wild in catsi if wild]) for catsi in cats]

[2, 2, 2]

In [54]:
cats[0]

[('12345761234567213456723145672341567234516723456172345612734562173456271345627314562734156273451627345',
  False),
 ('1236574123567412357641235746213574623157462351746235714623574162357412635742163574261357426315742635174263571426357',
  False),
 ('123764512374652137465231746523714652374165237461523746', False),
 ('12376541273654217365427136542731654273615427365142736512473652147365241736524713652473165247361524736',
  False),
 ('124653712463572146357241635724613572463157246351724635', False),
 ('12475631247536214753624175362471536247513624753162475312647532164753261475326417532647153264751326475',
  False),
 ('124763512746351276435127634512763541276534127654312764532176453271645327614532764153276451327645',
  False),
 ('125347612534672153467251346725314672534167253461725346127534612573461253746215374625137462531746253714625374162537412653742165374261537426513742653174265371426537',
  False),
 ('125637412536742153674251367425316742536174253671425367', False),
 ('125647312546731254763

In [55]:
cats[1]

[('12367541263754126735412675342167534261753426715342675134267531426753',
  False),
 ('12435761243567214356724135672431567243516724356172435612743562174356271435627413562743156274351627435',
  False),
 ('12457631245736214573624157362451736245713624573162457312645732164573261457326415732645173264571326457',
  False),
 ('12463571246375214637524163752461375246317524637152463712546372154637251463725416372546137254631725463',
  False),
 ('124736512743651273465127364512736541276354217635427163542761354276315427635142763512476352147635241763524716352476135247631524763',
  False),
 ('12536471235647123654712364572136457231645723614572364157236451723645',
  False),
 ('125643712654372165437261543726514372654137265431726543', False),
 ('125674312657432165743261574326517432657143265741326574', False),
 ('12573641257346215734625173462571346257314625734162573412657342165734261573426517342657134265731426573',
  False),
 ('12574361254736125437612543671245367214536724153672451367245316724536172453612745

In [56]:
cats[2]

[('12346571234675123476512347562134756231475623417562347156234751623475126347521634752613475263147526341752634715263471256347126534712635472163547261354726315472635147263541726354',
  False),
 ('12354761235467213546723154672351467235416723546172354612735462173546271354627315462735146273541527364',
  True),
 ('12364571236475213647523164752361475236417523647152364712536472153647251364725316472536147253641725364',
  False),
 ('1237465123745621374562317456237145623741562374516237451263745123674512376452137645231764523716452376145237641523764',
  False),
 ('1246573124567312457632145763241576324517632457163245761324576', False),
 ('12536741253764125374612534762153476251347625314762534176253471625347',
  False),
 ('1254637124563712465372146537241653724615372465137246531724653', False),
 ('125763412573642157364251736425713642573164257361425736', False),
 ('126357412365742136574231657423615742365174236571423657', False),
 ('1264735126743512673452167345261734526713452673145267341526734', False),

In [57]:
def get_perms(cat):
    cat = cat
    perms = [cat[i:i+7] for i in range(len(cat))]
    perms = [p for p in perms if p in permutations]
    return perms

In [58]:
cat = '12573642157364251736425713642573164257361325746'
cat

'12573642157364251736425713642573164257361325746'

In [59]:
cat[:-6] + '*' + cat[-5:] + cat[-6]

'12573642157364251736425713642573164257361*257463'

In [58]:
def get_string(cycles):
    used = []
    string = ''
    for cycle in cycles:
        cat, wild = get_concat(cycle)
        cycle = [permutations[i] for i in cycle]
        used.extend([p for p in permutations1 if p in cycle])
        if wild:
            cat = cat[:-6] + '*' + cat[-5:] + cat[-6]
            used.append('1'+cat[-6:])
        string = string + cat        
        #print(len(cat), len(string), len(used))
    unused = [p for p in permutations1 if p not in used]
    print(len(string), len(used), len(unused))
    string = string + ''.join(unused)
    print()
    return string

In [59]:
strings = [get_string(cycles) for cycles in all_cycles]
[len(s) for s in strings]

6713245 7162453
6415273 4162735
2162 82 38

4713256 7142563
3716245 7132456
2162 82 38

5416273 4152736
3714256 7132564
2162 82 38



[2428, 2428, 2428]

In [62]:
wild_permutations =  [''.join(x) for x in itertools.permutations(['1','2','3','4','5','6','7', '*'], 7)]
len(wild_permutations)

40320

In [63]:
def expand(perm):
    perms = [perm.replace('*', str(i)) for i in range(1,8)]
    perms = [p for p in perms if p in permutations]
    if len(perms) > 1:
        print(perm, perms)
    return perms[0]

def get_wild_perms(cat):
    perms = [cat[i:i+7] for i in range(len(cat))]
    perms = [p for p in perms if p in wild_permutations]
    perms = [expand(p) for p in perms]
    return perms    

In [64]:
get_wild_perms('1*276354')

['1427635', '1276354']

In [65]:
[len(s) for s in strings]

[2428, 2428, 2428]

In [66]:
perms = [get_wild_perms(s) for s in strings]
all_perms = [p for perm in perms for p in perm]

1234576 ['1234576', '1234576', '1234576', '1234576', '1234576', '1234576', '1234576']
2345761 ['2345761', '2345761', '2345761', '2345761', '2345761', '2345761', '2345761']
3457612 ['3457612', '3457612', '3457612', '3457612', '3457612', '3457612', '3457612']
4576123 ['4576123', '4576123', '4576123', '4576123', '4576123', '4576123', '4576123']
5761234 ['5761234', '5761234', '5761234', '5761234', '5761234', '5761234', '5761234']
7612345 ['7612345', '7612345', '7612345', '7612345', '7612345', '7612345', '7612345']
1234567 ['1234567', '1234567', '1234567', '1234567', '1234567', '1234567', '1234567']
3456721 ['3456721', '3456721', '3456721', '3456721', '3456721', '3456721', '3456721']
4567213 ['4567213', '4567213', '4567213', '4567213', '4567213', '4567213', '4567213']
5672134 ['5672134', '5672134', '5672134', '5672134', '5672134', '5672134', '5672134']
6721345 ['6721345', '6721345', '6721345', '6721345', '6721345', '6721345', '6721345']
7213456 ['7213456', '7213456', '7213456', '7213456', '

In [67]:
for p in permutations:
    if not p in all_perms:
        print(p)

In [68]:
for p in permutations1:
    for perm in perms:
        if not p in perm:
            print(p)

In [69]:
def finalize(strings):
    replace_dict = {
        "1": '🎅', 
        "2": '🤶', 
        "3": '🦌', 
        "4": '🧝', 
        "5": '🎄', 
        "6": '🎁', 
        "7": '🎀', 
        "*": '🌟',
    }
    ans = strings.copy()
    for i in range(3):
        for k,v in replace_dict.items():
            ans[i] = ans[i].replace(k, v)
    return ans

In [70]:
ans = finalize(strings)
ans

['🎅🤶🦌🧝🎄🎀🎁🎅🤶🦌🧝🎄🎁🎀🤶🎅🦌🧝🎄🎁🎀🤶🦌🎅🧝🎄🎁🎀🤶🦌🧝🎅🎄🎁🎀🤶🦌🧝🎄🎅🎁🎀🤶🦌🧝🎄🎁🎅🎀🤶🦌🧝🎄🎁🎅🤶🎀🦌🧝🎄🎁🤶🎅🎀🦌🧝🎄🎁🤶🎀🎅🦌🧝🎄🎁🤶🎀🦌🎅🧝🎄🎁🤶🎀🦌🧝🎅🎄🎁🤶🎀🦌🧝🎄🎅🎁🤶🎀🦌🧝🎄🎅🤶🦌🎁🎄🎀🧝🎅🤶🦌🎄🎁🎀🧝🎅🤶🦌🎄🎀🎁🧝🎅🤶🦌🎄🎀🧝🎁🤶🎅🦌🎄🎀🧝🎁🤶🦌🎅🎄🎀🧝🎁🤶🦌🎄🎅🎀🧝🎁🤶🦌🎄🎀🎅🧝🎁🤶🦌🎄🎀🧝🎅🎁🤶🦌🎄🎀🧝🎅🤶🎁🦌🎄🎀🧝🤶🎅🎁🦌🎄🎀🧝🤶🎁🎅🦌🎄🎀🧝🤶🎁🦌🎅🎄🎀🧝🤶🎁🦌🎄🎅🎀🧝🤶🎁🦌🎄🎀🎅🧝🤶🎁🦌🎄🎀🎅🤶🦌🎀🎁🧝🎄🎅🤶🦌🎀🧝🎁🎄🤶🎅🦌🎀🧝🎁🎄🤶🦌🎅🎀🧝🎁🎄🤶🦌🎀🎅🧝🎁🎄🤶🦌🎀🧝🎅🎁🎄🤶🦌🎀🧝🎁🎅🎄🤶🦌🎀🧝🎁🎅🤶🦌🎀🎁🎄🧝🎅🤶🎀🦌🎁🎄🧝🤶🎅🎀🦌🎁🎄🧝🤶🎀🎅🦌🎁🎄🧝🤶🎀🦌🎅🎁🎄🧝🤶🎀🦌🎁🎅🎄🧝🤶🎀🦌🎁🎄🎅🧝🤶🎀🦌🎁🎄🎅🤶🧝🎀🦌🎁🎄🤶🎅🧝🎀🦌🎁🎄🤶🧝🎅🎀🦌🎁🎄🤶🧝🎀🎅🦌🎁🎄🤶🧝🎀🦌🎅🎁🎄🤶🧝🎀🦌🎁🎅🎄🤶🧝🎀🦌🎁🎅🤶🧝🎁🎄🦌🎀🎅🤶🧝🎁🦌🎄🎀🤶🎅🧝🎁🦌🎄🎀🤶🧝🎅🎁🦌🎄🎀🤶🧝🎁🎅🦌🎄🎀🤶🧝🎁🦌🎅🎄🎀🤶🧝🎁🦌🎄🎅🎀🤶🧝🎁🦌🎄🎅🤶🧝🎀🎄🎁🦌🎅🤶🧝🎀🎄🦌🎁🤶🎅🧝🎀🎄🦌🎁🤶🧝🎅🎀🎄🦌🎁🤶🧝🎀🎅🎄🦌🎁🤶🧝🎀🎄🎅🦌🎁🤶🧝🎀🎄🦌🎅🎁🤶🧝🎀🎄🦌🎅🤶🎁🧝🎀🎄🦌🤶🎅🎁🧝🎀🎄🦌🤶🎁🎅🧝🎀🎄🦌🤶🎁🧝🎅🎀🎄🦌🤶🎁🧝🎀🎅🎄🦌🤶🎁🧝🎀🎄🎅🦌🤶🎁🧝🎀🎄🎅🤶🧝🎀🎁🦌🎄🎅🤶🎀🧝🎁🦌🎄🎅🤶🎀🎁🧝🦌🎄🎅🤶🎀🎁🦌🧝🎄🎅🤶🎀🎁🦌🎄🧝🎅🤶🎀🎁🎄🦌🧝🎅🤶🎀🎁🎄🧝🦌🎅🤶🎀🎁🧝🎄🦌🤶🎅🎀🎁🧝🎄🦌🤶🎀🎅🎁🧝🎄🦌🤶🎀🎁🎅🧝🎄🦌🤶🎀🎁🧝🎅🎄🦌🤶🎀🎁🧝🎄🎅🦌🤶🎀🎁🧝🎄🎅🤶🎄🦌🧝🎀🎁🎅🤶🎄🦌🧝🎁🎀🤶🎅🎄🦌🧝🎁🎀🤶🎄🎅🦌🧝🎁🎀🤶🎄🦌🎅🧝🎁🎀🤶🎄🦌🧝🎅🎁🎀🤶🎄🦌🧝🎁🎅🎀🤶🎄🦌🧝🎁🎅🤶🎀🎄🦌🧝🎁🎅🤶🎄🎀🦌🧝🎁🎅🤶🎄🦌🎀🧝🎁🤶🎅🎄🦌🎀🧝🎁🤶🎄🎅🦌🎀🧝🎁🤶🎄🦌🎅🎀🧝🎁🤶🎄🦌🎀🎅🧝🎁🤶🎄🦌🎀🧝🎅🎁🤶🎄🦌🎀🧝🎅🤶🎁🎄🦌🎀🧝🤶🎅🎁🎄🦌🎀🧝🤶🎁🎅🎄🦌🎀🧝🤶🎁🎄🎅🦌🎀🧝🤶🎁🎄🦌🎅🎀🧝🤶🎁🎄🦌🎀🎅🧝🤶🎁🎄🦌🎀🎅🤶🎄🎁🦌🎀🧝🎅🤶🎄🦌🎁🎀🧝🤶🎅🎄🦌🎁🎀🧝🤶🎄🎅🦌🎁🎀🧝🤶🎄🦌🎅🎁🎀🧝🤶🎄🦌🎁🎅🎀🧝🤶🎄🦌🎁🎀🎅🧝🤶🎄🦌🎁🎀🎅🤶🎄🎁🧝🎀🦌🎅🤶🎄🧝🎁🎀🦌🎅🤶🎄🧝🎀🎁🦌🎅🤶🎄🧝🎀🦌🎁🤶🎅🎄🧝🎀🦌🎁🤶🎄🎅🧝🎀🦌🎁🤶🎄🧝🎅🎀🦌🎁🤶🎄🧝🎀🎅🦌🎁🤶🎄🧝🎀🦌🎅🎁🤶🎄🧝🎀🦌🎅🤶🎁🎄🧝🎀🦌🤶🎅🎁🎄🧝🎀🦌🤶🎁🎅🎄🧝🎀🦌🤶🎁🎄🎅🧝🎀🦌🤶🎁🎄🧝🎅🎀🦌🤶🎁🎄🧝🎀🎅🦌🤶🎁🎄🧝🎀🎅🤶🎄🎁🎀🦌🧝🎅🤶🎄🎀🎁🦌🧝🤶🎅🎄🎀🎁🦌🧝🤶🎄🎅🎀🎁🦌🧝🤶🎄🎀🎅🎁🦌🧝🤶🎄🎀🎁🎅🦌🧝🤶🎄🎀

In [71]:
[len(a) for a in ans]

[2428, 2428, 2428]

In [73]:
import pandas as pd
sub = pd.DataFrame()
sub['schedule'] = ans
sub.to_csv('../subs/submission_2428_2.csv',index=False)
sub.head()

,schedule
0,🎅🤶🦌🧝🎄🎀🎁🎅🤶🦌🧝🎄🎁🎀🤶🎅🦌🧝🎄🎁🎀🤶🦌🎅🧝🎄🎁🎀🤶🦌🧝🎅🎄🎁🎀🤶🦌🧝🎄🎅🎁🎀🤶🦌🧝🎄...
1,🎅🤶🦌🎁🎀🎄🧝🎅🤶🎁🦌🎀🎄🧝🎅🤶🎁🎀🦌🎄🧝🎅🤶🎁🎀🎄🦌🧝🤶🎅🎁🎀🎄🦌🧝🤶🎁🎅🎀🎄🦌🧝🤶🎁🎀🎅...
2,🎅🤶🦌🧝🎁🎄🎀🎅🤶🦌🧝🎁🎀🎄🎅🤶🦌🧝🎀🎁🎄🎅🤶🦌🧝🎀🎄🎁🤶🎅🦌🧝🎀🎄🎁🤶🦌🎅🧝🎀🎄🎁🤶🦌🧝🎅...
